<a href="https://colab.research.google.com/github/Swapn2/PyTorch/blob/main/ANN_fashion_using_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
import torch
torch.manual_seed(42)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
df = pd.read_csv('/content/fashion-mnist_train.csv')

In [5]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,0,0,105,92,101,107,100,132,0,0,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,...,211,220,214,74,0,255,222,128,0,0,0,0,0,0,0,0,0,44,12,0,0,40,134,162,191,214,163,146,165,79,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,0,114,183,112,55,23,72,102,165,160,28,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,24,188,163,93,...,171,249,207,197,202,45,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22,21,25,69,52,45,74,39,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,46,0,21,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,187,189,...,230,237,229,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,116,112,136,147,144,121,102,63,0,0,0,0,0,0,0,0,0,0


In [16]:
x= df.iloc[:,1:]
y = df.iloc[:,0]

In [17]:
x

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,5,0,0,0,105,92,101,107,100,132,0,0,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,227,...,211,220,214,74,0,255,222,128,0,0,0,0,0,0,0,0,0,44,12,0,0,40,134,162,191,214,163,146,165,79,0,0,0,30,43,0,0,0,0,0
3,0,0,0,1,2,0,0,0,0,0,114,183,112,55,23,72,102,165,160,28,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,24,188,163,93,136,...,171,249,207,197,202,45,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22,21,25,69,52,45,74,39,3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,46,0,21,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,187,189,0,...,230,237,229,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,116,112,136,147,144,121,102,63,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59996,0,0,0,0,0,0,0,0,0,0,83,155,136,116,148,110,118,67,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,118,144,...,199,165,108,108,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32,159,6,91,0,0,85,159,0,73,0,0,0,0,0,0,0,0,0
59997,0,0,0,0,0,0,0,0,0,0,1,0,0,87,114,77,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,...,228,229,231,231,218,230,255,0,0,0,0,0,0,0,0,0,0,0,116,140,147,166,176,174,173,173,174,173,177,164,160,162,163,135,94,0,0,0,0,0
59998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = .2)

In [22]:
x_train = x_train/255.0
x_test = x_test/255

In [23]:
class CustomDataset(Dataset):
  def __init__(self,features, labels):
    self.features = torch.tensor(features ,dtype=torch.float32)
    self.labels = torch.tensor(labels , dtype = torch.long)

  def __len__(self):
    return len(self.features)
  def __getitem__(self,index):
    return self.features[index] , self.labels[index]

In [25]:
train_dataset = CustomDataset(x_train.values , y_train.values)
test_dataset = CustomDataset(x_test.values, y_test.values)

In [31]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset , batch_size = 32 , shuffle=True)
test_loader = DataLoader(test_dataset , batch_size = 32 , shuffle=False)

In [32]:
class MyNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features, 128 ),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64,10)   # by default softmax is implemented here no need to add
    )
  def forward(self, x):
    return self.model(x)

In [33]:
epochs = 100
learning_rate = .1


In [34]:
model = MyNN(x_train.shape[1])
model.to(device)  # change is necesaaary

loss = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr = learning_rate)



In [35]:
for epoch in range(epochs):

  total_epoch_loss = 0
  for batch_feature , batch_labels in train_loader:
    batch_feature , batch_labels = batch_feature.to(device), batch_labels.to(device)  # change is necessary
    # forward pass
    out = model(batch_feature)

    # loss
    batch_loss = loss(out,batch_labels)

    # back pass
    optimizer.zero_grad()
    batch_loss.backward()

    #
    optimizer.step()

    total_epoch_loss = total_epoch_loss + batch_loss.item()
  avg_loss= total_epoch_loss/len(train_loader)
  print(f'Epoch : {epoch +1}, Loss: {avg_loss}')

Epoch : 1, Loss: 2.3032918089230856
Epoch : 2, Loss: 2.3032478464444477
Epoch : 3, Loss: 2.3027933492660524
Epoch : 4, Loss: 2.301802049001058
Epoch : 5, Loss: 2.2968910732269285
Epoch : 6, Loss: 2.2186796112060545
Epoch : 7, Loss: 2.0468052858511605
Epoch : 8, Loss: 1.8023257341384888
Epoch : 9, Loss: 1.63158860484759
Epoch : 10, Loss: 1.5102546263535817
Epoch : 11, Loss: 1.4133748278220495
Epoch : 12, Loss: 1.3451295267740886
Epoch : 13, Loss: 1.292734536767006
Epoch : 14, Loss: 1.2535668264230093
Epoch : 15, Loss: 1.2094204972585043
Epoch : 16, Loss: 1.1753565673033397
Epoch : 17, Loss: 1.151647696296374
Epoch : 18, Loss: 1.127125209848086
Epoch : 19, Loss: 1.0912678266763687
Epoch : 20, Loss: 1.0590216521024705
Epoch : 21, Loss: 1.0264531909823418
Epoch : 22, Loss: 1.0041459119121234
Epoch : 23, Loss: 0.9831784882148107
Epoch : 24, Loss: 0.9672042619387309
Epoch : 25, Loss: 0.9398575189510981
Epoch : 26, Loss: 0.9251070677042007
Epoch : 27, Loss: 0.9142952985962232
Epoch : 28, Loss

In [36]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [39]:
total = 0
correct = 0
with torch.no_grad():
  for batch_feature , batch_label in test_loader:
    batch_feature , batch_labels = batch_feature.to(device), batch_label.to(device)  # change is necessary
    out = model(batch_feature)
    _,predict = torch.max(out ,1)
    total = total + batch_label.shape[0]
    correct = correct + (predict == batch_labels).sum().item()
print(correct/total)

0.7611666666666667
